## Using Gensim for Topic Modeling

We’re going to use the gensim implementations because they offer more functionality out of the box and then we’ll replicate that functionality with sklearn. Let’s first prepare the dataset we’ll be working with.


In [1]:
!pip install sastrawi
!pip install pyldavis
!pip install gensim==3.8.0

import nltk
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re 

import re
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')


     |████████████████████████████████| 215kB 8.7MB/s 
     |████████████████████████████████| 1.6MB 8.9MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=1da3206b33ee5bfc1fd3ac3739871748b5eda42b9c06514a1a638c88ce4797c9
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis
     |████████████████████████████████| 24.2MB 75.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!pip install gensim==3.8.0
import pkg_resources
pkg_resources.get_distribution("gensim").version


'3.6.0'

In [3]:
!git clone https://github.com/project303/dataset.git
  
article = open('dataset/Berita.txt', encoding="utf8").read().split('BERHENTI DISINI')
len(article)

Cloning into 'dataset'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


32

Clean the data from html tags with ``beautifulsoup``

In [4]:
article_clean = []
for text in article:
    text = BeautifulSoup(text, 'html.parser').getText()
    article_clean.append(text)
article = article_clean
print(article[0][:100])



Kroasia: Melawan Argentina adalah Pertandingan Termudah





Jakarta, CNN Indonesia -- Agung Rahma


Tokenize and clean stopwords

In [0]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [0]:
def tokenize_and_stem(text):
    stopwords = nltk.corpus.stopwords.words('indonesian')
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token) and token not in stopwords:
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [7]:
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in article:
    tokenized_data.append(tokenize_and_stem(text))

# Build a Dictionary - association word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
 
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
 
# Have a look at how the 20th document looks like: [(word_id, count), ...]
print(corpus[20])
# [(12, 3), (14, 1), (21, 1), (25, 5), (30, 2), (31, 5), (33, 1), (42, 1), (43, 2),  ...


[(2, 1), (3, 1), (11, 1), (19, 4), (22, 2), (24, 1), (26, 1), (44, 1), (50, 1), (164, 2), (183, 9), (196, 1), (223, 2), (230, 1), (252, 4), (274, 1), (280, 1), (284, 1), (309, 1), (314, 1), (335, 2), (341, 1), (404, 1), (431, 1), (434, 1), (452, 1), (465, 1), (474, 1), (480, 1), (485, 2), (489, 1), (500, 1), (504, 1), (520, 1), (523, 1), (534, 1), (538, 2), (546, 1), (547, 9), (552, 1), (596, 1), (597, 3), (599, 1), (619, 2), (676, 1), (845, 1), (904, 1), (914, 2), (927, 1), (932, 1), (967, 1), (1032, 1), (1038, 2), (1165, 2), (1174, 1), (1283, 1), (1397, 1), (1398, 8), (1399, 1), (1400, 1), (1401, 3), (1402, 1), (1403, 1), (1404, 2), (1405, 4), (1406, 1), (1407, 2), (1408, 3), (1409, 1), (1410, 1), (1411, 1), (1412, 3), (1413, 1), (1414, 4), (1415, 2), (1416, 1), (1417, 1), (1418, 1), (1419, 1), (1420, 1), (1421, 1), (1422, 1), (1423, 1), (1424, 2), (1425, 1), (1426, 1), (1427, 1), (1428, 1), (1429, 1), (1430, 1), (1431, 4), (1432, 1), (1433, 1), (1434, 1), (1435, 1), (1436, 1), (1437

In [0]:
NUM_TOPICS = 4

# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha = 'auto', eval_every=5)#, per_word_topics=True)
 
# Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

We’re going to run LDA and LSI (Latent Semantic Indexing AKA Latent Semantic Analysis) models, which implementation included in the gensim package.

Let’s now display the topics the two models have inferred:

In [9]:
print("LDA Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)
 
print("LSI Model:")
 
for idx in range(NUM_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

LDA Model:
Topic #0: 0.013*"persen" + 0.007*"indonesia" + 0.006*"jakarta" + 0.006*"duga" + 0.005*"diskon" + 0.005*"polisi" + 0.005*"laku" + 0.005*"lapor" + 0.005*"uang" + 0.004*"cnn"
Topic #1: 0.025*"persen" + 0.008*"main" + 0.006*"dunia" + 0.006*"tanding" + 0.006*"indonesia" + 0.006*"lawan" + 0.006*"lemah" + 0.006*"jakarta" + 0.005*"indeks" + 0.005*"dolar"
Topic #2: 0.006*"duga" + 0.005*"cnn" + 0.005*"lapor" + 0.005*"jakarta" + 0.005*"persen" + 0.005*"indonesia" + 0.005*"dunia" + 0.004*"namun" + 0.004*"orang" + 0.004*"laku"
Topic #3: 0.012*"persen" + 0.006*"oknum" + 0.006*"duga" + 0.006*"novel" + 0.006*"indonesia" + 0.005*"jakarta" + 0.005*"diskon" + 0.004*"minus" + 0.004*"jenderal" + 0.004*"aman"
LSI Model:
Topic #0: 0.731*"persen" + 0.216*"lemah" + 0.171*"dolar" + 0.165*"minus" + 0.133*"indeks" + 0.128*"bunga" + 0.116*"kuat" + 0.111*"uang" + 0.104*"as" + 0.103*"dagang"
Topic #1: -0.385*"novel" + -0.311*"oknum" + -0.300*"duga" + -0.271*"jenderal" + -0.165*"kpk" + -0.142*"main" + -0.1

Let’s now put the models to work and transform unseen documents to their topic distribution:

In [10]:
text = "Pertandingan berjalan dengan seru. Tim lawan berhasil dikalahkan dengan skor 1-0."
bow = dictionary.doc2bow(tokenize_and_stem(text))

print(lda_model[bow]) 
print(lsi_model[bow])
print(bow)

[(0, 0.05583), (1, 0.8432569), (2, 0.053605795), (3, 0.047307327)]
[(0, 0.13315407974835333), (1, -0.3590914551263508), (2, 0.8065685310165087), (3, 0.039665126690103754)]
[(19, 1), (46, 1), (75, 1), (137, 1), (404, 1), (454, 1), (930, 1)]


The LDA result can be interpreted as a distribution over topics.
Gensim offers a simple way of performing similarity queries using topic models.

In [11]:
from gensim import similarities
 
lda_index = similarities.MatrixSimilarity(lda_model[corpus])
 
# Let's perform some queries
similarities = lda_index[lda_model[bow]]
# Sort the similarities
similarities = sorted(enumerate(similarities), key=lambda item: -item[1])
 
# Top most similar documents:
print(similarities[:10])
 
# Let's see what's the most similar document
document_id, similarity = similarities[0]
print(article[document_id][:1000])

[(1, 0.99426746), (4, 0.99426746), (5, 0.99426746), (6, 0.99426746), (7, 0.99426746), (10, 0.99426746), (12, 0.99426746), (13, 0.99426746), (14, 0.99426746), (18, 0.99426746)]



Jakarta, CNN Indonesia -- Nilai tukar mata uang negara-negara di kawasan Asia terpantau melemah di hadapan dolar AS pada hari ini, Rabu (13/6), tepat sehari sebelum The Federal Reserve, bank sentral Amerika Serikat (AS), mengumumkan keputusan tingkat suku bunga acuannya.

Sebelumnya, The Fed memberi sinyal akan menaikkan tingkat suku bunga acuannya sebanyak tiga kali pada tahun ini dan kenaikan kedua diperkirakan terjadi pada Juni ini. 

Pelemahan tertinggi terjadi pada mata uang Korea Selatan, yaitu won hingga 0,82 persen. Diikuti, rupee India minus 0,19 persen, yen Jepang minus 0,18 persen, dan baht Thailand minus 0,13 persen. 

Lalu, ringgit Malaysia melemah 0,12 persen, peso Filipina minus 0,12 persen, renmimbi China minus 0,04 persen, dolar Hong Kong minus 0,02 persen, dan dolar Singapura minus 0,01 perse

Notice how the factors corresponding to each component (topic) add up to 1. That’s not a coincidence. Indeed, LDA considers documents as being generated by a mixture of the topics. The purpose of LDA is to compute how much of the document was generated by which topic. 

LDA is an iterative algorithm. Here are the two main steps:

   - In the initialization stage, each word is assigned to a random topic.
   - Iteratively, the algorithm goes through each word and reassigns the word to a topic taking into consideration:
        - What’s the probability of the word belonging to a topic
        - What’s the probability of the document to be generated by a topic

Due to these important qualities, we can visualize LDA results easily. We’re going to use a specialized tool called PyLDAVis:

In [12]:
import pyLDAvis.gensim
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
panel

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.054387 -0.000610       1        1  48.478966
0     -0.034148 -0.009270       2        1  27.962366
2     -0.016580  0.019134       3        1  15.099586
3     -0.003658 -0.009254       4        1   8.459080, topic_info=     Category        Freq       Term       Total  loglift  logprob
254   Default   24.000000       duga   24.000000  30.0000  30.0000
835   Default   14.000000      novel   14.000000  29.0000  29.0000
839   Default   16.000000      oknum   16.000000  28.0000  28.0000
180   Default  111.000000     persen  111.000000  27.0000  27.0000
1476  Default   18.000000     diskon   18.000000  26.0000  26.0000
802   Default   12.000000   jenderal   12.000000  25.0000  25.0000
767   Default   11.000000       aman   11.000000  24.0000  24.0000
165   Default   17.000000      minus   17.000000  23.0000  23.0000
26    Default   36.000000    jakarta   36.000000  22.0000  22.0000
375   Default   13.000000      orang   13.000000  21.0000  21.0000
24    Default   38.000000  indonesia   38.000000  20.0000  20.0000
45    Default   20.000000      lapor   20.000000  19.0000  19.0000
11    Default   26.000000        cnn   26.000000  18.0000  18.0000
281   Default   19.000000       laku   19.000000  17.0000  17.0000
305   Default   13.000000      rumah   13.000000  16.0000  16.0000
1398  Default   15.000000      balap   15.000000  15.0000  15.0000
55    Default   19.000000     polisi   19.000000  14.0000  14.0000
817   Default   10.000000        kpk   10.000000  13.0000  13.0000
264   Default    9.000000      hukum    9.000000  12.0000  12.0000
75    Default   22.000000    tanding   22.000000  11.0000  11.0000
946   Default   13.000000       golf   13.000000  10.0000  10.0000
1223  Default    9.000000      vonis    9.000000   9.0000   9.0000
1371  Default    8.000000      woods    8.000000   8.0000   8.0000
1211  Default    8.000000     sidang    8.000000   7.0000   7.0000
54    Default   12.000000      pasal   12.000000   6.0000   6.0000
82    Default   26.000000       uang   26.000000   5.0000   5.0000
660   Default   14.000000     terima   14.000000   4.0000   4.0000
19    Default   19.000000      hasil   19.000000   3.0000   3.0000
730   Default    7.000000      mudik    7.000000   2.0000   2.0000
50    Default   16.000000      namun   16.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
254    Topic4    3.173224       duga   24.140894   0.4408  -5.1325
1476   Topic4    2.493432     diskon   18.890615   0.4449  -5.3736
165    Topic4    2.307624      minus   17.695112   0.4329  -5.4510
1223   Topic4    1.337753      vonis    9.362343   0.5242  -5.9962
817    Topic4    1.405149        kpk   10.021628   0.5053  -5.9471
825    Topic4    0.803410      libat    5.310042   0.5814  -6.5061
264    Topic4    1.266738      hukum    9.458188   0.4595  -6.0508
660    Topic4    1.656636     terima   14.158770   0.3244  -5.7824
885    Topic4    0.886394     ungkap    6.116229   0.5384  -6.4078
562    Topic4    0.921736    uruguay    6.627616   0.4972  -6.3687
94     Topic4    1.594106         as   14.324086   0.2743  -5.8209
159    Topic4    1.407079       mata   12.039566   0.3232  -5.9457
180    Topic4    6.206705     persen  111.960495  -0.4226  -4.4616
24     Topic4    2.969899  indonesia   38.102798  -0.0818  -5.1987
26     Topic4    2.826827    jakarta   36.022343  -0.0751  -5.2481
1803   Topic4    1.642482    belanja   16.664066   0.1529  -5.7910
204    Topic4    1.393165      saham   12.926471   0.2422  -5.9556
114    Topic4    2.006048      dolar   24.700012  -0.0407  -5.5911
82     Topic4    2.060249       uang   26.174883  -0.0720  -5.5644
186    Topic4    1.357999         pt   12.937261   0.2158  -5.9812
155    Topic4    1.964729      lemah   25.854399  -0.1072  -5.6119
55     Topic4    1.679980     polisi   19.412727   0.0228  -5.7684
1106   Topic4  